<a href="https://colab.research.google.com/github/pedrodeparis/Recommendation-Systems/blob/main/Recomenda%C3%A7%C3%A3o_de_Filme_com_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163004 sha256=655dbec5b38ab2add0429426fb4f43c3c6619bb9411ef9de0b0fce1d6597add0
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import os
import numpy as np
import pandas as pd
from google.colab import files

In [3]:
%%time
_ = files.upload()

Saving ratings.parquet to ratings.parquet
CPU times: user 1.19 s, sys: 227 ms, total: 1.42 s
Wall time: 1min 39s


In [4]:
df_ratings = pd.read_parquet('ratings.parquet')
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [5]:
%%time
_ = files.upload()

Saving movies.parquet to movies.parquet
CPU times: user 617 ms, sys: 75.8 ms, total: 693 ms
Wall time: 55 s


In [6]:
df_items = pd.read_parquet('movies.parquet')
df_items.set_index('item_id', inplace=True)
df_items.tail()

,title,genres
item_id,,
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama
3951,Two Family House (2000),Drama
3952,"Contender, The (2000)",Drama|Thriller


In [7]:
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [8]:
train_size = 0.8
# Ordenar por timestamp
df_ratings = df_ratings.sort_values(by='timestamp', ascending=True)

# Definindo train e valid sets
df_train_set, df_valid_set = np.split(df_ratings, [ int(train_size*df_ratings.shape[0]) ])

# Redefinindo nomes para userID e itemID
df_train_set = df_train_set.rename({'user_id': 'userID', 'item_id': 'itemID'}, axis=1)
df_valid_set = df_valid_set.rename({'user_id': 'userID', 'item_id': 'itemID'}, axis=1)

print ('Train size: ', df_train_set.shape)
print ('Valid size: ', df_valid_set.shape)

Train size:  (800167, 4)
Valid size:  (200042, 4)


In [9]:
df_train_set

,userID,itemID,rating,timestamp
1000138,6040,858,4,956703932
1000153,6040,2384,4,956703954
999873,6040,593,5,956703954
1000007,6040,1961,4,956703977
1000192,6040,2019,5,956703977
...,...,...,...,...
314102,1875,802,4,975768718
314151,1875,892,4,975768719
314073,1875,440,4,975768738
314225,1875,509,4,975768738


In [10]:
from surprise import Dataset, Reader
def convert_train_valid_sets(df_train_set:pd.DataFrame, df_valid_set:pd.DataFrame):
  reader = Reader(rating_scale=(1, 5))
  train_set = (
      Dataset
      .load_from_df(df_train_set[['userID', 'itemID', 'rating']], reader)
      .build_full_trainset()
  )

  valid_set = (
      Dataset
      .load_from_df(df_valid_set[['userID', 'itemID', 'rating']], reader)
      .build_full_trainset()
      .build_testset()
  )

  return train_set, valid_set

train_set, valid_set = convert_train_valid_sets(df_train_set, df_valid_set)

In [11]:
train_set

In [12]:
from surprise import KNNWithMeans

sim_options = {
    "name": "pearson_baseline",
    "user_based": False,  # compute similarities between items
}
model = KNNWithMeans(k=40, sim_options=sim_options, verbose=True)
model

In [13]:
%%time
model.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
CPU times: user 16.9 s, sys: 645 ms, total: 17.5 s
Wall time: 17.5 s


In [14]:
user_id = df_valid_set['userID'].sample().tolist()[0]
item_id = df_valid_set['itemID'].sample().tolist()[0]

model.predict(uid=user_id, iid=item_id)

Prediction(uid=1338, iid=1, r_ui=None, est=4.5047685566675515, details={'actual_k': 40, 'was_impossible': False})

In [15]:
%%time
df_valid_set['prediction'] = df_valid_set.apply(
    lambda x: model.predict(uid=x['userID'], iid=x['itemID']).est,
    axis=1
)

CPU times: user 24 s, sys: 74.4 ms, total: 24.1 s
Wall time: 24.7 s


In [16]:
df_valid_set.tail()

,userID,itemID,rating,timestamp,prediction
825793,4958,2399,1,1046454338,2.561691
825438,4958,1407,5,1046454443,3.552168
825724,4958,3264,4,1046454548,3.270295
825731,4958,2634,3,1046454548,2.754026
825603,4958,1924,4,1046454590,2.584696


In [17]:
def recommend_n_items(model, user_id, item_ids:np.array, n=20):
  df_predictions = pd.DataFrame(columns=['item_id', 'score'])
  for item_id in item_ids:
    prediction = model.predict(uid=user_id, iid=item_id).est
    df_predictions.loc[df_predictions.shape[0]] = [item_id, prediction]

  user_predictions = (
      df_predictions
      .sort_values(by='score', ascending=False)
      .head(n)
      .set_index('item_id')
  )
  return user_predictions

user_id = 1875
recommendable_items = df_items.index.values
recommend_n_items(model, user_id, recommendable_items, n=5)

,score
item_id,
1830.0,5.0
989.0,5.0
3522.0,5.0
3881.0,5.0
1471.0,5.0


In [18]:
item_id = 1     # Toy Story
item_id = 1356  # Star Trek: First Contact
item_id = 260   # Star Wars: Episode IV - A New Hope
# item_id = 3578  # Gladiator

def get_item_k_neighbors(model, item_id, k=10):
  iid = model.trainset.to_inner_iid(item_id)
  neighbor_iids = model.get_neighbors(iid, k)
  item_ids = [model.trainset.to_raw_iid(iid) for iid in neighbor_iids]
  return item_ids

k = 10
title = df_items.loc[item_id]['title']
print (f'{k} vizinhos mais próximos de "{title}" (ID = {item_id})')
item_ids = get_item_k_neighbors(model, item_id, k)
df_items[df_items.index.isin(item_ids)]

10 vizinhos mais próximos de "Star Wars: Episode IV - A New Hope (1977)" (ID = 260)


,title,genres
item_id,,
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
2640,Superman (1978),Action|Adventure|Sci-Fi
2716,Ghostbusters (1984),Comedy|Horror
3507,"Odd Couple, The (1968)",Comedy


In [ ]:
from tqdm import tqdm
model_name = 'itemknn'
n = 20
df_recommendations = pd.DataFrame()
catalog = df_items.index.values
for user_id in tqdm(df_valid_set['userID'].unique()):
  user_known_items = df_train_set.query('userID == @user_id')['itemID'].unique()
  recommendable_items = np.array(list(set(catalog)-set(user_known_items)))

  user_recommendations = recommend_n_items(model, user_id, recommendable_items, n).reset_index(drop=False)
  user_recommendations['user_id'] = user_id
  df_recommendations = pd.concat([df_recommendations, user_recommendations])

df_recommendations['model'] = model_name
df_recommendations = df_recommendations.merge(
    df_valid_set,
    left_on=['user_id', 'item_id'],
    right_on=['userID', 'itemID'],
    how='left'
)
df_recommendations = df_recommendations[['model', 'user_id', 'item_id', 'rating', 'score']]
df_recommendations['rating'] = df_recommendations['rating'].fillna(0)
df_rec_bkp = df_recommendations.copy()
df_recommendations.tail()


 10%|▉         | 171/1783 [13:07<2:16:37,  5.09s/it]

In [ ]:
df_recommendations['y_score'] = df_recommendations.apply(lambda x: {'item_id': x['item_id'], 'score': x['score']}, axis=1)
df_recommendations = df_recommendations.groupby(['model', 'user_id']).agg({'y_score': list}).reset_index(drop=False)
df_recommendations.tail()


In [ ]:
df_predictions = df_valid_set.rename({'userID': 'user_id', 'itemID': 'item_id'}, axis=1)
df_predictions['y_true'] = df_predictions.apply(lambda x: {'item_id': x['item_id'], 'rating': x['rating']}, axis=1)
df_predictions = df_predictions.groupby('user_id').agg({'y_true': list}).reset_index(drop=False)
df_predictions = df_predictions.merge(df_recommendations, on='user_id', how='inner')
df_predictions.tail()

In [ ]:
df_predictions.to_parquet(f'valid_{model_name}.parquet', index=None)